In [4]:
from google.cloud import pubsub

def publish_message(topic_name, data):
    """Publishes a message to a Pub/Sub topic with the given data."""
    pubsub_client = pubsub.Client()
    topic = pubsub_client.topic(topic_name)

    message_id = topic.publish(data)

    print('Message {} published.'.format(message_id))
    
def receive_message(topic_name, subscription_name):
    """Receives a message from a pull subscription."""
    pubsub_client = pubsub.Client()
    topic = pubsub_client.topic(topic_name)
    subscription = topic.subscription(subscription_name)

    # Change return_immediately=False to block until messages are
    # received.
    results = subscription.pull(return_immediately=True)

    print('Received {} messages.'.format(len(results)))

    # Acknowledge received messages. If you do not acknowledge, Pub/Sub will
    # redeliver the message.
    if results:
        subscription.acknowledge([ack_id for ack_id, message in results])
        
    return results

In [1]:
import orca

from zone_model import datasources
from zone_model import variables
from zone_model import models

transition_models = ['simple_jobs_transition',
                     'simple_households_transition',
                     'simple_residential_units_transition',
                     'simple_non_residential_units_transition']

relocation_models = ['simple_jobs_relocation',
                     'simple_households_relocation']

choice_models = ['elcm1',
                 'hlcm1', 'hlcm2', 'hlcm3', 'hlcm4',
                 'rdplcm1',
                 'nrdplcm1']

orca.run(transition_models + relocation_models + choice_models)

Model template is: zone. Operating on zones with zone_id.
Running step 'simple_jobs_transition'
Running jobs transition with 0.50 percent growth rate
1199753 agents before transition
1205752 agents after transition
Time to execute step 'simple_jobs_transition': 0.23 s
Running step 'simple_households_transition'
Running households transition with 0.50 percent growth rate
1085402 agents before transition
1090829 agents after transition
Time to execute step 'simple_households_transition': 0.38 s
Running step 'simple_residential_units_transition'
Running residential_units transition with 0.50 percent growth rate
1182965 agents before transition
1188880 agents after transition
Time to execute step 'simple_residential_units_transition': 0.21 s
Running step 'simple_non_residential_units_transition'
Running non_residential_units transition with 0.50 percent growth rate
893642 agents before transition
898110 agents after transition
Time to execute step 'simple_non_residential_units_transition':

/home/janowicz/pulls/urbansim/urbansim/urbanchoice/interaction.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  alts_sample['join_index'] = np.repeat(choosers.index.values, SAMPLE_SIZE)


Choice mode is individual, so utilizing lottery choices.
There are 0 unplaced agents.
Time to execute step 'hlcm1': 3.57 s
Running step 'hlcm2'
12168 agents are making a choice.
There are 1182965.0 total available units     and 12168 total choosers     but there are 0 overfull alternatives
    for a total of 104304.0 temporarily empty units     in 219 alternatives total in the region
There are 12168 total movers for this LCM
There are 0 unplaced agents.
Time to execute step 'hlcm2': 0.24 s
Running step 'hlcm3'
8053 agents are making a choice.
There are 1182965.0 total available units     and 8053 total choosers     but there are 0 overfull alternatives
    for a total of 100189.0 temporarily empty units     in 219 alternatives total in the region
There are 8053 total movers for this LCM
There are 0 unplaced agents.
Time to execute step 'hlcm3': 0.17 s
Running step 'hlcm4'
4003 agents are making a choice.
There are 1182965.0 total available units     and 4003 total choosers     but ther

In [7]:
change_sets = orca.get_injectable('change_sets')

In [8]:
for cs in change_sets:
    cs.export()

Message 121602442865940 published.
Message 121663590046523 published.
Message 121549545120646 published.
Message 121668734417734 published.
Message 121548667493186 published.
Message 121668734417773 published.
Message 121612664338848 published.
Message 121612664338875 published.
Message 121707169110862 published.
Message 121557761995680 published.
Message 121707169110896 published.
Message 121589874448657 published.
Message 121545608783743 published.


In [10]:
publish_message('changeset123abc', cs[0].export())

Message 121580083126208 published.


In [17]:
results = receive_message('changeset123abc', 'mypubsub123abc')

Received 1 messages.


In [10]:
results

[(u'QV5AEkw-BURJUytDCypYEU4EISE-MD5FU0RQBhYsXUZIUTcZCGhRDk9eIz81IChFEAAIFAV8fXFbV3VfXRoHUQ0ZcnxkJzxcEQJTTFp6VVkRDXptXFc4UA8YdHticGNaEwUEQFF9d7Sl94NpZho9XxJLLD5-NTBF',
  <google.cloud.pubsub.message.Message at 0x7fa99d014290>)]

In [18]:
import pandas as pd
pd.read_msgpack(results[0][1].data)

chooser_id
884         22
1110        76
2637       119
3410        92
3939         1
4589        77
4974        93
4975        88
5278       222
5510        20
5784        50
6130         6
6685        70
6765       119
6869       118
7163        70
7254        74
7357       103
7685        50
8164        41
8331       119
8501        26
8641        88
9217         3
9247       142
9400        70
10124       20
10694      142
11037       70
11762       26
          ... 
1088489    103
1088620    151
1088628     77
1088633    185
1088675    103
1088848     84
1089080      1
1089181     85
1089248     85
1089471    160
1089538     88
1089622    113
1089681    119
1089730    125
1089823    103
1089829     19
1089899     77
1089959      4
1090150    103
1090156    103
1090231    103
1090237    119
1090449      1
1090454    125
1090488     16
1090540    103
1090737    103
1090894     69
1090900    109
1091058    171
dtype: int64

In [7]:
for cs in orca.get_injectable('change_sets'):
    print cs.operation
    print cs.table
    print cs.model_step
    print cs.data

added
jobs
simple_jobs_transition
job_id
875603      875603
1258107    1258107
111738      111738
834747      834747
824690      824690
376495      376495
218417      218417
761786      761786
759474      759474
1179608    1179608
260007      260007
538655      538655
805566      805566
966885      966885
289188      289188
852923      852923
923352      923352
926981      926981
1052280    1052280
449188      449188
1199349    1199349
1143911    1143911
1130724    1130724
8647          8647
851462      851462
580804      580804
967684      967684
768449      768449
782590      782590
892589      892589
            ...   
734205      734205
1137806    1137806
1289232    1289232
106362      106362
890468      890468
464332      464332
537361      537361
390501      390501
1139305    1139305
93679        93679
1317849    1317849
912767      912767
291427      291427
200742      200742
28447        28447
733564      733564
541293      541293
335102      335102
1067992    1067992
597389   